# Introduction

## Lab2: Train a Convolutional Neural Network (CNN).

In this Lab session we will learn how to train a CNN from scratch for classifying MNIST digits.


In [2]:
# import necessary libraries
import torch
import torchvision
from torchvision import transforms as T
import torch.nn.functional as F
import sklearn

### Define LeNet

![network architecture](https://www.researchgate.net/profile/Lucijano-Berus/publication/329891470/figure/fig1/AS:707347647307776@1545656229128/Architecture-of-LeNet-5-a-Convolutional-Neural-Network-for-digits-digits-recognition-An.ppm)

Here we are going to define our first CNN which is **LeNet** in this case. This architecture has been introduced and is detailed in [this article](http://vision.stanford.edu/cs598_spring07/papers/Lecun98.pdf). To construct a LeNet we will be using some convolutional layers followed by some fully-connected layers. The convolutional layers can be simply defined using `torch.nn.Conv2d` module of `torch.nn` package. Details can be found [here](https://pytorch.org/docs/stable/nn.html#conv2d). Moreover, we will use pooling operation to reduce the size of convolutional feature maps. For this case we are going to use `torch.nn.functional.max_pool2d`. Details about maxpooling can be found [here](https://pytorch.org/docs/stable/nn.html#max-pool2d)

Differently from our previous Lab, we will use a Rectified Linear Units (ReLU) as activation function with the help of `torch.nn.functional.relu`, replacing `torch.nn.Sigmoid`. Details about ReLU can be found [here](https://pytorch.org/docs/stable/nn.html#id26).


In [3]:
class LeNet(torch.nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()

        # input channel = ?, output channels = ?, kernel size = ?
        # input image size = (?, ?), image output size = (?, ?)
        self.conv1 = torch.nn.Conv2d(1, 6, 5, padding=2)

        # input channel = ?, output channels = ?, kernel size = ?
        # input image size = (?, ?), output image size = (?, ?)
        self.conv2 = torch.nn.Conv2d(6, 16, 5)

        # input dim = ? ( H x W x C), output dim = ?
        self.fc1 = torch.nn.Linear(16 * 5 * 5, 120)

        # input dim = ?, output dim = ?
        self.fc2 = torch.nn.Linear(120, 84)

        # input dim = ?, output dim = ?
        self.fc3 = torch.nn.Linear(84, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        # Max Pooling with kernel size = ?
        # output size = (?, ?)
        x = F.max_pool2d(x, 2)

        x = F.relu(self.conv2(x))
        # Max Pooling with kernel size = ?
        # output size = (?, ?)
        x = F.max_pool2d(x, 2)

        # flatten the feature maps into a long vector
        x = x.view(x.shape[0], -1)

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x

### Define cost function


In [4]:
def get_cost_function():
    cost_function = torch.nn.CrossEntropyLoss()
    return cost_function

### Define the optimizer

We will use SGD with learning rate-lr, weight_decay=wd and momentum=momentum


In [5]:
def get_optimizer(net, lr, wd, momentum):
    optimizer = torch.optim.SGD(
        net.parameters(), lr=lr, weight_decay=wd, momentum=momentum
    )
    return optimizer

### Train and test functions


In [6]:
def test(net, data_loader, cost_function, device="cuda:0"):
    samples = 0.0
    cumulative_loss = 0.0
    cumulative_accuracy = 0.0

    net.eval()  # Strictly needed if network contains layers which has different behaviours between train and test
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(data_loader):
            # Load data into GPU
            inputs = inputs.to(device)
            targets = targets.to(device)

            # Forward pass
            outputs = net(inputs)

            # Apply the loss
            loss = cost_function(outputs, targets)

            # Better print something
            samples += inputs.shape[0]
            cumulative_loss += (
                loss.item()
            )  # Note: the .item() is needed to extract scalars from tensors
            _, predicted = outputs.max(1)
            cumulative_accuracy += predicted.eq(targets).sum().item()

    return cumulative_loss / samples, cumulative_accuracy / samples * 100


def train(net, data_loader, optimizer, cost_function, device="cuda:0"):
    samples = 0.0
    cumulative_loss = 0.0
    cumulative_accuracy = 0.0

    net.train()  # Strictly needed if network contains layers which has different behaviours between train and test
    for batch_idx, (inputs, targets) in enumerate(data_loader):
        # Load data into GPU
        inputs = inputs.to(device)
        targets = targets.to(device)

        # Forward pass
        outputs = net(inputs)

        # Apply the loss
        loss = cost_function(outputs, targets)

        # Reset the optimizer

        # Backward pass
        loss.backward()

        # Update parameters
        optimizer.step()
        optimizer.zero_grad()

        # Better print something, no?
        samples += inputs.shape[0]
        cumulative_loss += loss.item()
        _, predicted = outputs.max(1)
        cumulative_accuracy += predicted.eq(targets).sum().item()

    return cumulative_loss / samples, cumulative_accuracy / samples * 100

### Define the function that fetches a data loader that is then used during iterative training.

We will learn a new thing in this function as how to Normalize the inputs given to the network.

**_Why Normalization is needed_**?

To have nice and stable training of the network it is recommended to normalize the network inputs between \[-1, 1\].

**_How it can be done_**?

This can be simply done using `torchvision.transforms.Normalize()` transform. Details can be found [here](https://pytorch.org/docs/stable/torchvision/transforms.html#torchvision.transforms.Normalize).


In [7]:
def get_data(batch_size, test_batch_size=256):
    # Prepare data transformations and then combine them sequentially
    transform = list()
    transform.append(T.ToTensor())  # converts Numpy to Pytorch Tensor
    transform.append(
        T.Normalize(mean=[0.5], std=[0.5])
    )  # Normalizes the Tensors between [-1, 1]
    transform = T.Compose(transform)  # Composes the above transformations into one.

    # Load data
    full_training_data = torchvision.datasets.MNIST(
        "./data", train=True, transform=transform, download=True
    )
    test_data = torchvision.datasets.MNIST(
        "./data", train=False, transform=transform, download=True
    )

    # Create train and validation splits
    num_samples = len(full_training_data)
    training_samples = int(num_samples * 0.5 + 1)
    validation_samples = num_samples - training_samples

    training_data, validation_data = torch.utils.data.random_split(
        full_training_data, [training_samples, validation_samples]
    )

    # Initialize dataloaders
    train_loader = torch.utils.data.DataLoader(training_data, batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(
        validation_data, test_batch_size, shuffle=False
    )
    test_loader = torch.utils.data.DataLoader(test_data, test_batch_size, shuffle=False)

    return train_loader, val_loader, test_loader

### Wrapping everything up

Finally, we need a main function which initializes everything + the needed hyperparameters and loops over multiple epochs (printing the results).


In [8]:
"""
Input arguments
  batch_size: Size of a mini-batch
  device: GPU where you want to train your network
  weight_decay: Weight decay co-efficient for regularization of weights
  momentum: Momentum for SGD optimizer
  epochs: Number of epochs for training the network
"""


def main(
    batch_size=128,
    device="cuda:0",
    learning_rate=0.01,
    weight_decay=0.000001,
    momentum=0.9,
    epochs=50,
):
    device = torch.device(device)
    train_loader, val_loader, test_loader = get_data(batch_size)

    # TODO for defining LeNet-5
    net = LeNet().to(device)
    optimizer = get_optimizer(net, learning_rate, weight_decay, momentum)
    cost_function = get_cost_function()

    print("Before training:")
    train_loss, train_accuracy = test(net, train_loader, cost_function)
    val_loss, val_accuracy = test(net, val_loader, cost_function)
    test_loss, test_accuracy = test(net, test_loader, cost_function)

    print(
        "\t Training loss {:.5f}, Training accuracy {:.2f}".format(
            train_loss, train_accuracy
        )
    )
    print(
        "\t Validation loss {:.5f}, Validation accuracy {:.2f}".format(
            val_loss, val_accuracy
        )
    )
    print("\t Test loss {:.5f}, Test accuracy {:.2f}".format(test_loss, test_accuracy))
    print("-----------------------------------------------------")

    for e in range(epochs):
        train_loss, train_accuracy = train(net, train_loader, optimizer, cost_function)
        val_loss, val_accuracy = test(net, val_loader, cost_function)
        print("Epoch: {:d}".format(e + 1))
        print(
            "\t Training loss {:.5f}, Training accuracy {:.2f}".format(
                train_loss, train_accuracy
            )
        )
        print(
            "\t Validation loss {:.5f}, Validation accuracy {:.2f}".format(
                val_loss, val_accuracy
            )
        )
        print("-----------------------------------------------------")

    print("After training:")
    train_loss, train_accuracy = test(net, train_loader, cost_function)
    val_loss, val_accuracy = test(net, val_loader, cost_function)
    test_loss, test_accuracy = test(net, test_loader, cost_function)

    print(
        "\t Training loss {:.5f}, Training accuracy {:.2f}".format(
            train_loss, train_accuracy
        )
    )
    print(
        "\t Validation loss {:.5f}, Validation accuracy {:.2f}".format(
            val_loss, val_accuracy
        )
    )
    print("\t Test loss {:.5f}, Test accuracy {:.2f}".format(test_loss, test_accuracy))
    print("-----------------------------------------------------")

Lets train!


In [9]:
main()

Before training:
	 Training loss 0.01807, Training accuracy 8.76
	 Validation loss 0.00907, Validation accuracy 8.74
	 Test loss 0.00923, Test accuracy 8.64
-----------------------------------------------------
Epoch: 1
	 Training loss 0.00938, Training accuracy 59.16
	 Validation loss 0.00109, Validation accuracy 91.66
-----------------------------------------------------
Epoch: 2
	 Training loss 0.00154, Training accuracy 93.80
	 Validation loss 0.00067, Validation accuracy 94.74
-----------------------------------------------------
Epoch: 3
	 Training loss 0.00098, Training accuracy 96.05
	 Validation loss 0.00048, Validation accuracy 96.16
-----------------------------------------------------
Epoch: 4
	 Training loss 0.00072, Training accuracy 97.14
	 Validation loss 0.00035, Validation accuracy 97.35
-----------------------------------------------------
Epoch: 5
	 Training loss 0.00057, Training accuracy 97.78
	 Validation loss 0.00033, Validation accuracy 97.38
------------------

Using the proper metric from sklearn, check which character is most frequently confused with which: can you explain why ?


In [10]:
# Using the proper metric from sklearn, check which character is most frequently confused with which: can you explain why ?

The LeNet5 architecture can also be implemented using the sequential API ([see documentation ](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html)). Reimplement it with this API.


## Experiments

- Implement adaptive early stopping: if the validation loss did not decrease for K consecutive epochs, stop training.
- Change dataset in order to evaluate the LeNet5 network on cifar10 dataset. You can have a look at the pytorch to easily access the cifar10 dataset.
- Try to improve performance with:
  - data-augmentation
  - dropout
- Implement the resnet18 architecture using the Resnet18 class from pytorch.
